In [1]:
import mmcv

mmcv.collect_env()

from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector
from mmrotate.models import build_detector

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
import sys, os

sys.path.append("./protos")

import grpc
# import messaging_pb2
# import messaging_pb2_grpc

import os
import sys
import random
import cv2
import time
import torch

class ImageProcessingLayer:
    def __init__(
        self,
        mock=True,
        mock_image_path=None,
        mock_num_samples=10,
        mock_wait_time=1,
    ):
        self.mock = mock

        if not mock:
            return

        self.mock_wait_time = mock_wait_time

        if mock_image_path is None:
            mock_image_path = "../data/demo.jpg" #jpg matters

        self._mock_img_full = np.asarray(Image.open(mock_image_path))
        self._output_dim = (1000, 1000)
        diag_len = np.sqrt(self._output_dim[0] ** 2 + self._output_dim[1] ** 2)
#         self._gcps_pixels = self._generate_random_gcps(
#             self._mock_img_full, mock_num_samples, padding=(diag_len, diag_len)
#         )

        self._gcps_pixels = np.array([(8000, 3000), (9000, 3000), (9000, 4000), (11000, 5000)])
        
        # maybe convert from pixels to lat/lon here

        self._path_pixels = self._build_path_pixels(self._gcps_pixels)

    def _generate_random_gcps(self, img, num_samples, padding=(0, 0)):
        return np.random.randint(
            padding,
            high=(img.shape[0] - padding[0], img.shape[1] - padding[1]),
            size=(num_samples, 2),
        )

    def _build_path_pixels(self, gcps):
        delta = np.diff(gcps, axis=0)
        directions = delta / np.linalg.norm(delta, axis=1).reshape(-1, 1)
        angles = np.arctan2(directions.T[1], directions.T[0]) * 180 / np.pi
        delta_angles = np.append(np.diff(angles), 0)

        path = []

        for t1, t2, angle, delta_angle in zip(gcps, gcps[1:], angles, delta_angles):
            steps = np.linalg.norm(t2 - t1) / 90
            line = np.linspace(t1, t2, steps.astype("uint32"), dtype="uint32")
            path.extend([np.array([x, y, angle]) for x, y in line])

            if delta_angle == 0:
                continue

            if len(line) == 0:
                continue

            interpolated_angles = np.linspace(angle, angle + delta_angle, 3)
            path.extend(
                [
                    np.array([line[-1][0], line[-1][1], theta])
                    for theta in interpolated_angles
                ]
            )

        return path

    def _next_image(self):
        if self.mock_wait_time > 0:
            time.sleep(self.mock_wait_time)

        sample_diag = np.sqrt(self._output_dim[0] ** 2 + self._output_dim[1] ** 2)

        for x, y, theta in self._path_pixels:
            sample = self._crop_around(
                self._mock_img_full, (y, x), (sample_diag, sample_diag)
            )
            rotated_img = self._center_crop(
                rotate(sample, -theta, reshape=False), self._output_dim
            )
            yield rotated_img

    def _crop_around(self, img, center, dim):
        dim = np.array(dim).astype("uint32")
        x = int(center[1] - dim[1] // 2)
        y = int(center[0] - dim[0] // 2)
        return img[y : y + dim[0], x : x + dim[1]]

    def _center_crop(self, img, dim):
        return img[
            img.shape[0] // 2 - dim[0] // 2 : img.shape[0] // 2 + dim[0] // 2,
            img.shape[1] // 2 - dim[1] // 2 : img.shape[1] // 2 + dim[1] // 2,
        ]

    def run(self, img=None):
        if not self.mock:
            assert img is not None, "Image cannot be None"
            return img

        return self._next_image()


class ObjectDetectionLayer:
    def __init__(
        self, config_file=None, checkpoint_file=None, device="cuda", min_confidence=0.3
    ):
        if config_file is None:
            config_file = "../examples/oriented_rcnn_r50_fpn_1x_dota_le90.py"
        if checkpoint_file is None:
            checkpoint_file = "../examples/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth"

        self.config_file = config_file
        self.checkpoint_file = checkpoint_file
        self.device = device

        self.model = self._load_model()
        self.min_confidence = min_confidence

    def _load_model(self):
        
        config = mmcv.Config.fromfile(self.config_file)
        config.model.pretrained = None

        model = build_detector(config.model)
        checkpoint = load_checkpoint(
            model, self.checkpoint_file, map_location=self.device
        )

        model.CLASSES = checkpoint["meta"]["CLASSES"]
        model.cfg = config
        model.to(self.device)
        model = model.eval()

        return model

    def _get_bboxes_pixels(self, img):
        vehicle_classes = [
            i for i, c in enumerate(self.model.CLASSES) if "vehicle" in c
        ]
        inference = inference_detector(self.model, img)
        bboxes = [inference[index] for index in vehicle_classes]

        bboxes = np.concatenate(bboxes, axis=0)
        bboxes = bboxes[bboxes[:, 5] > self.min_confidence]

        # the bboxes are in a weird polygonal format, so we convert them to rectangles
        rect_bboxes = (
            np.array(
                [
                    bboxes[:, 1] - bboxes[:, 2] // 2,
                    bboxes[:, 1] + bboxes[:, 2] // 2,
                    bboxes[:, 0] - bboxes[:, 2] // 2,
                    bboxes[:, 0] + bboxes[:, 3],
                    100 * bboxes[:, -1],  # confidence score
                ]
            )
            .astype(int)
            .T
        )

        # follows the format of x0, x1, y0, y1, confidence
        return rect_bboxes

    def run(self, img):
        result = self._get_bboxes_pixels(img)

        # convert pixels to lat/lon here
        return result

    
    
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    
class OpenCVObjectDetectionLayer:
    def __init__(
        self, config_file=None, weights_file=None, classes_file = None, device=torch.device('cpu'), min_confidence=0.3
    ):
        self.config_file = config_file
        self.weights_file = weights_file
        self.classes_file = classes_file
        self.device = device

        self.net = self._load_model()
        self.min_confidence = min_confidence

    def _load_model(self):
        net = cv2.dnn.readNet(self.weights_file, self.config_file)

        return net

    def _get_bboxes_pixels(self, img):
        image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        Width = image.shape[1]
        Height = image.shape[0]
        scale = 0.00392

        classes = None

        
        with open(classes_file, 'r') as f:
            classes = [line.strip() for line in f.readlines()]

        COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

        blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

        self.net.setInput(blob)

        outs = self.net.forward(get_output_layers(self.net))

        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.5
        nms_threshold = 0.4


        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])


        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)


        bboxes_with_confidence = []
        for i in indices:
            try:
                box = boxes[i]

            except:
                i = i[0]
                box = boxes[i]


            x = box[0]
            y = box[1]
            w = box[2]
            h = box[3]
            
            if x < 0:
                x = 0
            if y < 0:
                y = 0
            
            bboxes_with_confidence.append(np.array((x, x+w, y, y+h, 100*confidences[i])))
            
        # follows the format of x0, x1, y0, y1, confidence
        return np.array(bboxes_with_confidence).astype(int)
    
    def _bbox_pixels_to_gps(self, bboxes, gps_corners, img_dim):
        top_left_gps, top_right_gps, bot_left_gps, bot_right_gps = gps_corners
        right_vec = (top_right_gps - top_left_gps) / img_dim[1]
        bot_vec = (bot_left_gps - top_left_gps) / img_dim[0]

        transformation = np.array([[right_vec[0], bot_vec[0]], [right_vec[1], bot_vec[1]]])

        def transform(bbox):
            return transformation @ bbox + top_left_gps
            
        return np.array([transform(bbox) for bbox in bboxes])
    
    def _convert_bbox_to_radial_representation(self, bboxes):
        """
            Given a bbox in the format of x0, x1, y0, y1, confidence.
            Returns a bbox in the format of x, y, r, confidence
        """

        radii = np.sqrt((bboxes[:, 1] - bboxes[:, 0]) ** 2 + (bboxes[:, 3] - bboxes[:, 2]) ** 2)

        return np.array([
            (bboxes[:, 0] + bboxes[:, 1]) / 2,
            (bboxes[:, 2] + bboxes[:, 3]) / 2,
            radii,
            bboxes[:, 4]
        ]).T

    def run(self, img):
        bboxes_pixels = self._get_bboxes_pixels(img)
        
        return bboxes_pixels  # remove the pixels bboxes later
    

class MavlinkInterfaceLayer:
    def __init__(self, protos_path="protos"):
        self.protos_path = protos_path
        self.channel = grpc.insecure_channel("localhost:50051")
        self.stub = messaging_pb2_grpc.MessagingServiceStub(self.channel)
        pass

    def run(self, bboxes):
        if len(bboxes) == 0:
            return

        print(bboxes)

/home/matt/RT-Flight/testing/RT-env/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
img_layer = ImageProcessingLayer(mock_wait_time=1)


weights_file = "/home/matt/RT-Flight/yolo-test/yolov3-aerial.weights"
classes_file = "/home/matt/RT-Flight/yolo-test/aerial-darknet/data/aerial.names"
config_file = "/home/matt/RT-Flight/yolo-test/aerial-darknet/cfg/yolov3-aerial.cfg"

obj_layer = OpenCVObjectDetectionLayer(config_file=config_file, weights_file=weights_file, classes_file=classes_file, device='cpu')
# mav_layer = MavlinkInterfaceLayer()

lat_center = 29.643946
lon_center = -82.355659

lat_mile = 0.0144927536231884
lon_mile = 0.0181818181818182
lat_min = lat_center - (15 * lat_mile)
lat_max = lat_center + (15 * lat_mile)
lon_min = lon_center - (15 * lon_mile)
lon_max = lon_center + (15 * lon_mile)

for img in img_layer.run():
    bboxes = obj_layer.run(img)
#     mav_layer.run(bboxes)

    for bbox in bboxes:
        bbox = bbox.astype(float)
        bbox[0] = float(float(bbox[0]) / 200 * ( lat_max - lat_min ) + lat_min)
        bbox[1] = float(float(bbox[1]) / 200 * ( lon_max - lon_min ) + lon_min)
        output = str(bbox[:4])[1:-1]
        print(output)
        
#         response = mav_layer.stub.SendData(messaging_pb2.DataRequest(data=output))

/home/matt/RT-Flight/testing/RT-env/lib/python3.8/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (111695860 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


 29.7939 -81.9602 822.     876.    
 30.3287 -81.2811 881.     928.    
 29.5809 -82.2384 822.     875.    
 30.1157 -81.5538 883.     927.    
 29.9026 -81.8211 884.     926.    
 29.4266 -82.492  829.     880.    
 29.6874 -82.0857 886.     924.    
 29.4657 -82.3475 881.     929.    
 31.4809 -79.8902 770.     851.    
 31.3483 -80.0293 691.     774.    
 31.2266 -80.1929 604.     698.    
 31.1026 -80.3511 526.     619.    
 30.8853 -80.6211 904.     968.    
 30.9766 -80.5093 449.     540.    
 30.757  -80.7793 824.     890.    
 30.8526 -80.6702 374.     462.    
 30.6266 -80.9347 747.     812.    
 30.7287 -80.8311 298.     381.    
 30.5005 -81.0902 666.     735.    
 30.6005 -80.9947 222.     304.    
 30.37   -81.2457 588.     659.    
 30.4744 -81.1502 144.     223.    
 30.2418 -81.3984 505.     587.    
 30.3505 -81.3111  65.     148.    
 30.1113 -81.5593 428.     507.    
 31.5439 -79.9011 818.     894.    
 30.2222 -81.4584   0.      68.    
 31.4113 -79.9311 742.     8